In [2]:
import os
import pandas as pd
import numpy as np

# ---------------------------------------------------
# 1. Load Data and Initialize Directories
# ---------------------------------------------------

FILE_PATH = 'C:/Users/danie/Nextcloud/Coding/Masterthesis/data/raw/Socio_Economic_Accounts.xlsx'
OUTPUT_DIR = 'C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/price_data'
SHEET_NAME = 'DATA'

def load_data(file_path, sheet_name):
    """Load data from the specified Excel sheet."""
    return pd.read_excel(file_path, sheet_name=sheet_name)

def create_output_directory(directory_path):
    """Ensure the output directory exists."""
    os.makedirs(directory_path, exist_ok=True)

# ---------------------------------------------------
# 2. Calculate Percentage Price Changes for Price Indices
# ---------------------------------------------------

def calculate_percentage_price_changes(price_data, year_columns):
    """
    Calculate the yearly percentage changes of prices.
    """
    pct_price_changes = price_data[['country', 'variable', 'code']].copy()
    for i in range(1, len(year_columns)):
        current_year = year_columns[i]
        previous_year = year_columns[i - 1]
        pct_price_changes[f'{current_year}_pct_change'] = (
            (price_data[current_year] - price_data[previous_year]) / price_data[previous_year]
        ) * 100

    return pct_price_changes

# ---------------------------------------------------
# 3. Calculate Mean-Adjusted Volatility for Each Price Index
# ---------------------------------------------------

def calculate_price_volatility(df, variable):
    """
    Calculate the standard deviation of yearly percentage price changes
    for the specified price index variable using the mean-adjusted volatility formula.
    """
    price_data = df[df['variable'] == variable]
    year_columns = df.columns[4:]  # Assuming first 4 columns are metadata
    pct_price_changes = calculate_percentage_price_changes(price_data, year_columns)
    
    # Calculate mean of yearly percentage changes across the period
    pct_change_columns = [col for col in pct_price_changes.columns if col.endswith('_pct_change')]
    pct_price_changes['mean_pct_change'] = pct_price_changes[pct_change_columns].mean(axis=1)

    # Calculate the mean-adjusted volatility
    pct_price_changes['price_volatility'] = np.sqrt(((pct_price_changes[pct_change_columns].sub(pct_price_changes['mean_pct_change'], axis=0)) ** 2).mean(axis=1))
    
    # Prepare the output with necessary columns
    volatility_data = pct_price_changes[['country', 'code', 'price_volatility']]
    
    return volatility_data

# ---------------------------------------------------
# 4. Map, Adjust, Extend Volatility Data with New Countries
# ---------------------------------------------------

def map_and_adjust_volatility(df, variable):
    """
    Adjust the volatility data for the given price index variable by mapping two-letter country codes,
    setting volatility to 0 for excluded countries, and adding missing countries if necessary.
    """
    # Define excluded and missing countries
    excluded_countries = {"TWN"}  # Taiwan should be excluded
    additional_countries = ["SA", "ZA", "AR", "FIGW1"]  # Countries to add with zero volatility

    # Calculate price volatility for the specified variable
    volatility_df = calculate_price_volatility(df, variable)

    # Verify if 'price_volatility' column exists
    if volatility_df is None or 'price_volatility' not in volatility_df.columns:
        print(f"Error: Missing 'price_volatility' column in the DataFrame for {variable}.")
        return None

    # Convert SEA to FIGARO format (two-letter codes)
        "ESP": "ES", "FRA": "FR", "HRV": "HR", "ITA": "IT", "CYP": "CY", "LVA": "LV", "LTU": "LT", "LUX": "LU", 
        "HUN": "HU", "MLT": "MT", "NLD": "NL", "AUT": "AT", "POL": "PL", "PRT": "PT", "ROU": "RO", "SVN": "SI", 
        "SVK": "SK", "FIN": "FI", "SWE": "SE", "ARG": "AR", "AUS": "AU", "BRA": "BR", "CAN": "CA", "CHE": "CH", 
        "CHN": "CN", "IDN": "ID", "IND": "IN", "JPN": "JP", "KOR": "KR", "MEX": "MX", "NOR": "NO", "RUS": "RU", 
        "SAU": "SA", "TUR": "TR", "GBR": "GB", "USA": "US", "ZAF": "ZA"}
    volatility_df['country'] = volatility_df['country'].replace(sea_to_figaro_map)

    # Exclude countries like Taiwan
    volatility_df = volatility_df[~volatility_df['country'].isin(excluded_countries)]

    # Add additional countries with zero volatility for each industry sector (56 sectors assumed)
    industries = volatility_df['code'].unique()  # Extract all industry codes from existing data
    for country_code in additional_countries:
        for industry in industries:
            volatility_df = pd.concat([
                volatility_df,
                pd.DataFrame({'country': [country_code], 'code': [industry], 'price_volatility': [0]})
            ], ignore_index=True)

    # Create a combined column with `{ISO-code}_{NACE}` format
    volatility_df['sector'] = volatility_df['country'] + "_" + volatility_df['code']

    # Drop the separate `country` and `code` columns
    volatility_df = volatility_df[['sector', 'price_volatility']]

    # Sort by combined sector label for consistency
    volatility_df = volatility_df.sort_values(by='sector').reset_index(drop=True)
    return volatility_df

# ---------------------------------------------------
# 5. Save Price Volatility to CSV
# ---------------------------------------------------

def save_volatility_to_csv(volatility_df, variable, output_dir):
    """Save the calculated volatility to a CSV file in the output directory."""
    if volatility_df is not None:
        output_file = os.path.join(output_dir, f"{variable}_volatility.csv")
        volatility_df.to_csv(output_file, index=False)
        print(f"{variable} volatility data saved to: {output_file}")
    else:
        print(f"Error: No data to save for {variable}.")

# ---------------------------------------------------
# 6. Main Process Function to Handle All Price Indices
# ---------------------------------------------------

def main():
    # Load data
    df = load_data(FILE_PATH, SHEET_NAME)
    
    # Ensure output directory exists
    create_output_directory(OUTPUT_DIR)

    # Process volatility for each specified price index and save
    for variable in ['GO_PI', 'II_PI', 'VA_PI']:
        adjusted_volatility_df = map_and_adjust_volatility(df, variable)
        save_volatility_to_csv(adjusted_volatility_df, variable, OUTPUT_DIR)

# ---------------------------------------------------
# Execute the main function
# ---------------------------------------------------

if __name__ == "__main__":
    main()


GO_PI volatility data saved to: C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/price_data\GO_PI_volatility.csv
II_PI volatility data saved to: C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/price_data\II_PI_volatility.csv
VA_PI volatility data saved to: C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/price_data\VA_PI_volatility.csv
